### **STEP-1**. AI Solution 및 Instance 등록을 위한 준비 작업
&#x1F600; **등록 할 AI Contents 의 experimental_plan.yaml 를 alo/solution/ 에 준비해 둡니다.**

&#x1F600; **가상 환경을 만들어 두고, ipykernel 을 제작해 둡니다.**     

1. 본 jupyter notebook 실행 전에 alo/main.py 파일이 \
   존재하는 위치에서 아래 명령어들을 순차 실행합니다.
> conda create -n {ENV-NAME} python=3.10 \
> conda init bash \
> conda activate {ENV-NAME} \
> python main.py \
> pip install ipykernel \
> pip install requests \
> python -m ipykernel install --user --name {ENV-NAME} --display-name {IPYKERNEL-NAME}

&#x1F600; **아래 STEP들을 하나씩 실행시키면서, <u>< 사용자 입력 ></u>이라고 주석 표기된 내용을 적절히 변경해주세요.**     

----

### **STEP-2**. AI Solution INPUT setting

In [1]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
# 제공 cis-workspace 작업 시 로그인 불필요 
user_input = {
    
    # workspace 이름 
    'WORKSPACE_NAME': "cis",
    
    # 시스템 URI
    'URI': "https://aic-web-kic.aidxlge.com/", 
    
    # ECR에 올라갈 컨테이너 URI TAG 
    # [참고] release-2.1.3 에선 latest만 허용 
    'ECR_TAG': 'latest', 
    
    # scripts/creating_ai_solution/image/ 밑에 UI에 표시될 아이콘 이미지 파일 (ex. icon.png) 를 업로드 해주세요. 
    # 이후 해당 아이콘 파일 명을 아래에 기록 해주세요.
    'ICON_FILE': 'icon.svg'
}

SOLUTION_NAME = "customer-index"

> 시스템 로그인

In [2]:
# 시각화 용 모듈 install 
!pip install tabulate

# s3 용 모듈 install 
!pip install boto3 botocore GitPython

import sys
import os

# 현재 jupyternotebook kernerl 의 python pip list 를 requirements.txt 로 저장 합니다.
my_env_path = sys.exec_prefix + "/bin/"
assert os.path.isdir(my_env_path)
!{my_env_path}pip list --format=freeze > requirements.txt

In [3]:
# Instantiate AirflowRegisterer
import os
from register_utils import AirflowRegisterer
registerer = AirflowRegisterer(user_input)
registerer.solution_name = SOLUTION_NAME

/home/jovyan/airflow_test/alo2.2_tcr/


#### **STEE-2-2**. AI Solution Name 을 AI Conductor 에 등록합니다. 
&#x1F600; 이름이 등록되면 본 jupyter 노트북 과정이 끝날 때까지 변경이 어려 울 수 있습니다. \
변경이 필요할 경우 <b> STEP-2-1 </b> 부터 다시 실행하여 주시기 바랍니다. 
 

> 기존에 workspace에 존재하는 solution 이름을 조회하여, 사용자가 입력하는 이름이 유효한지 (고유한지) 확인합니다.

In [4]:
# 학습 용 solution_metadata.yaml 셋팅 
###################
pipeline = 'train'
###################
# 받아온 workspace 정보 기반으로 solution_metadata.yaml 셋팅 
registerer.set_yaml(pipeline = pipeline)


 << solution_metadata.yaml >> generated. - current version: v1


----

#### **STEP-3**. Train 용 Sample data, train artifacts, model, icon 등록

In [5]:
# ----------------
#  user input
# ----------------

# bucket 설정 
S3_BUCKET = 's3-an2-cis-dev-data' # options : ['s3-an2-cis-dev-data', 's3-a2-cis-prod-data'] # 

# prefix 설정
TRAIN_DATA_PREFIX = f'data/{SOLUTION_NAME}/train_data/' # must starts with : 'data/ ... '  
TRAIN_ARTIFACTS_PREFIX = f'data/{SOLUTION_NAME}/train_artifacts/' # must starts with : 'data/ ... '
ICON_PREFIX = f'data/{SOLUTION_NAME}/icon/'

train 용 s3 upload 실행

In [6]:
assert TRAIN_DATA_PREFIX.startswith('data'), ValueError("prefix needs to be start with data/")
assert TRAIN_ARTIFACTS_PREFIX.startswith('data'), ValueError("prefix needs to be start with data/")
assert ICON_PREFIX.startswith('data'), ValueError("prefix needs to be start with data/")

# data prefix 등록
registerer.set_s3_directly(bucket_name=S3_BUCKET,
                           prefix=TRAIN_DATA_PREFIX)

# s3 데이터 업로드
# 이전에 이미 해당 s3 경로에 존재하던 데이터는 지워집니다.
registerer.s3_upload_data()

# model / artifacts prefix 등록 
registerer.set_s3_directly(bucket_name=S3_BUCKET,
                           prefix=TRAIN_ARTIFACTS_PREFIX)

# s3 model, artifacts 업로드 
registerer.s3_upload_artifacts()

registerer.s3_upload_icon(icon_prefix=TRAIN_DATA_PREFIX) 


[INFO] AWS S3 access check: OK

[INFO] Start uploading data into S3 from local folder:
 /home/jovyan/airflow_test/alo2.2_tcr/input/train/

[INFO] Deleted pre-existing S3 object: data/customer-index/train_data/

[INFO] Deleted pre-existing S3 object: data/customer-index/train_data/inference_data/test/iris.csv

Success uploading into S3: 
s3-an2-cis-dev-data/data/customer-index/train_data/train/iris.csv

Success updating solution_metadata.yaml - << dataset_uri >> info / pipeline: train

[INFO] AWS S3 access check: OK

[INFO] Start uploading << train artifacts >> into S3 from local folder:
 /home/jovyan/airflow_test/alo2.2_tcr/.TEMP_ARTIFACTS_PATH/

[INFO] Deleted pre-existing S3 object: data/customer-index/train_artifacts/

[INFO] Deleted pre-existing S3 object: data/customer-index/train_artifacts/inference_artifacts.tar.gz

Success uploading into S3: 
s3-an2-cis-dev-data/data/customer-index/train_artifacts/train_artifacts.tar.gz

Success updating solution_metadata.yaml - << artifact_ur

----

#### **STEP-4**. Train Docker Container 제작

> ALO 작업 폴더를 현재 노트북 경로로 가져옵니다. \
> DOCKERFILE을 셋팅합니다. 

In [7]:
# ALO 작업 폴더 가져오기 
registerer.set_alo()

# DOCKERFILE setting ~ pipeline : tarain 기준으로 설정 되어 있는 상태
registerer.set_docker_contatiner()

[INFO] copy from " /home/jovyan/airflow_test/alo2.2_tcr/main.py "  -->  " /home/jovyan/airflow_test/alo2.2_tcr/airflow_registerer/alo/ " 
[INFO] copy from " /home/jovyan/airflow_test/alo2.2_tcr/src "  -->  " /home/jovyan/airflow_test/alo2.2_tcr/airflow_registerer/alo/src " 
[INFO] copy from " /home/jovyan/airflow_test/alo2.2_tcr/assets "  -->  " /home/jovyan/airflow_test/alo2.2_tcr/airflow_registerer/alo/assets " 
[INFO] copy from " /home/jovyan/airflow_test/alo2.2_tcr/alolib "  -->  " /home/jovyan/airflow_test/alo2.2_tcr/airflow_registerer/alo/alolib " 
[INFO] copy from " /home/jovyan/airflow_test/alo2.2_tcr/requirements.txt "  -->  " /home/jovyan/airflow_test/alo2.2_tcr/airflow_registerer/alo/ " 

 Success ALO directory setting.
Success DOCKERFILE setting. 
 - pipeline: train


> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [8]:
## ECR 등록

# "042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cis-prod/cis-workspace-user/"
ECR_BASE_URL = "042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cis-dev/cis-workspace-user/"
TRAIN_ECR_URI = os.path.join(ECR_BASE_URL,SOLUTION_NAME,'train') 

## buildah login 실행 (docker in docker) - sudo 권한 필요 
tags = [
    "Key=Company,Value=LGE",
    "Key=Owner,Value=IC360",
    "Key=HQ,Value=CDO",
    "Key=Division,Value=CDO",
    "Key=Infra Region,Value=KIC",
    "Key=Service Mode,Value=DE",
    "Key=Cost Type,Value=COMPUTING",
    "Key=Project,Value=CIS",
    "Key=Sub Project,Value=CISM",
    "Key=System,Value=AIDX"
]

registerer.set_ecr_directly(
    ecr_uri=TRAIN_ECR_URI,
    docker=False,
    tags=[]
)
# registerer.set_aws_ecr(docker=False, tags=tags) 

p1:  ['aws', 'ecr', 'get-login-password', '--region', 'ap-northeast-2']
[INFO] target AWS ECR url: 
042969618971.dkr.ecr.ap-northeast-2.amazonaws.com
p2:  ['sudo', 'buildah', 'login', '--username', 'AWS', '--password-stdin', '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com']
[INFO] AWS ECR | docker login result: 
 Login Succeeded!

[INFO] Target AWS ECR repository: 
ecr-repo-an2-cis-dev/cis-workspace-user/customer-index/train
Failed to AWS ECR create-repository:
 + Command '['aws', 'ecr', 'create-repository', '--region', 'ap-northeast-2', '--repository-name', 'ecr-repo-an2-cis-dev/cis-workspace-user/customer-index/train', '--image-scanning-configuration', 'scanOnPush=true']' returned non-zero exit status 254.
skip creating ecr : check authority or ecr-uri correctly


> Docker Build \
> ECR에 docker image를 push 합니다. \
> solution_metadata.yaml에 container uri를 넣어줍니다.

In [9]:
# docker build & push
registerer.build_docker()
registerer.docker_push()
registerer.set_container_uri() 

STEP 1/15: FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye
STEP 2/15: RUN apt-get update
Get:1 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:2 http://deb.debian.org/debian-security bullseye-security InRelease [48.4 kB]
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Get:4 http://deb.debian.org/debian bullseye/main amd64 Packages [8068 kB]
Get:5 http://deb.debian.org/debian-security bullseye-security/main amd64 Packages [269 kB]
Get:6 http://deb.debian.org/debian bullseye-updates/main amd64 Packages [18.8 kB]
Fetched 8564 kB in 1s (7672 kB/s)
Reading package lists...
STEP 3/15: RUN apt-get install -y apt-utils
Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  apt-utils
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 439 kB of archives.
After this operation, 1192 kB of additional disk space will be used.
Get:1 http://deb.debian.org

debconf: delaying package configuration, since apt-utils is not installed


Fetched 439 kB in 0s (12.2 MB/s)
Selecting previously unselected package apt-utils.
(Reading database ... 7040 files and directories currently installed.)
Preparing to unpack .../apt-utils_2.2.4_amd64.deb ...
Unpacking apt-utils (2.2.4) ...
Setting up apt-utils (2.2.4) ...
STEP 4/15: RUN apt-get install -y --no-install-recommends          build-essential          wget          ca-certificates          git          gcc     && rm -rf /var/lib/apt/lists/*
Reading package lists...
Building dependency tree...
Reading state information...
ca-certificates is already the newest version (20210119).
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu bzip2 cpp cpp-10 dpkg-dev
  g++ g++-10 gcc-10 git-man libasan6 libatomic1 libbinutils libbrotli1
  libc-dev-bin libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0 libctf0
  libcurl3-gnutls libdpkg-perl liberror-perl libgcc-10-dev libgdbm-compat4
  libgomp1 libisl23 libitm1 libldap-2.4-2 liblsan0 libmp

debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.32.1 /usr/local/share/perl/5.32.1 /usr/lib/x86_64-linux-gnu/perl5/5.32 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl-base /usr/lib/x86_64-linux-gnu/perl/5.32 /usr/share/perl/5.32 /usr/local/lib/site_perl) at /usr/share/perl5/Debconf/FrontEnd/Readline.pm line 7, <> line 56.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Selecting previously unselected package libgdbm-compat4:amd64.
Preparing to unpack .../01-libgdbm-compat4_1.19-2_amd64.deb ...
Unpacking libgdbm-compat4:amd64 (1.19-2) ...
Selecting previously unselected package libperl5.32:amd64.
Preparing to unpack .../02-libperl5.32_5.32.1-4+deb11u3_amd64.deb ...
Unpacking libperl5.32:amd64 (5.32.1-4+deb11u3) ...
Selecting previously unselected package perl.
Preparing to unpack .../03-perl_5.32.1-4+deb11u3_amd64.deb ...
Unpacking perl (5.32.1-4+deb11u3) ...
Selecting previously unselected package bzip2.
Preparing to unpack .../04-bzip2_1.0.8-4_amd64.deb ...
Unpacking bzip2 (1.0.8-4) ...
Selecting previously unselected package libpsl5:amd64.
Preparing to unpack .../05-libpsl5_0.21.0-1.2_amd64.deb ...
Unpacking libpsl5:amd64 (0.21.0-1.2) ...
Selecting previously unselected package wget.
Preparing to unpack .../06-wget_1.21-1+deb11u1_amd64.deb ...
Unpacking wget (1.21-1+deb11u1) ...
Selecting previously unselected package xz-utils.
Preparing to unpack 

...

STEP 8/15: ENV LC_ALL=C.UTF-8
STEP 9/15: ENV PYTHONUNBUFFERED=TRUE
STEP 10/15: ENV PYTHONDONTWRITEBYTECODE=TRUE
STEP 11/15: ENV SOLUTION_PIPELINE_MODE='train'
STEP 12/15: ENV PATH="/framework:${PATH}"
STEP 13/15: COPY /alo /framework
STEP 14/15: WORKDIR /framework
STEP 15/15: CMD ["python", "main.py"]
COMMIT 042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cis-dev/cis-workspace-user/customer-index/train:latest


Getting image source signatures
Copying blob sha256:0baf2321956a506afcddaafe217bc852e4c56a9640530b1b2f98b3378d4b6173
Copying blob sha256:e00d2ab597e7579cfd8251f047918d0bbfbd07230dad84a2f2a863fa0b5de84d
Copying blob sha256:2c73ee9daea8b487fd2035729eb8dc53c69d46dcab0b0a45114fe83a788a7faf
Copying blob sha256:b8ef0148a6d0f9aefbe116049b8a263aca65af0fa7c52072da94ef4434ca6263
Copying blob sha256:f9c504e80eedd8d7bd1fb3cfc88fcbc51b0f154844cae65d26939511f4a60d51
Copying blob sha256:fc78f86ac216321c7406cfd8d702a6ea5eaa665cf0a8b0d0785d3a2d1a0ee4be
Copying config sha256:d9e03d395ae0377c6d8d2cc8a279c365b33773d26603d8b328b2aed85759b4fc
Writing manifest to image destination
Storing signatures


--> d9e03d395ae
Successfully tagged 042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cis-dev/cis-workspace-user/customer-index/train:latest
d9e03d395ae0377c6d8d2cc8a279c365b33773d26603d8b328b2aed85759b4fc


Getting image source signatures
Copying blob sha256:0baf2321956a506afcddaafe217bc852e4c56a9640530b1b2f98b3378d4b6173
Copying blob sha256:2c73ee9daea8b487fd2035729eb8dc53c69d46dcab0b0a45114fe83a788a7faf
Copying blob sha256:b8ef0148a6d0f9aefbe116049b8a263aca65af0fa7c52072da94ef4434ca6263
Copying blob sha256:f9c504e80eedd8d7bd1fb3cfc88fcbc51b0f154844cae65d26939511f4a60d51
Copying blob sha256:e00d2ab597e7579cfd8251f047918d0bbfbd07230dad84a2f2a863fa0b5de84d
Copying blob sha256:fc78f86ac216321c7406cfd8d702a6ea5eaa665cf0a8b0d0785d3a2d1a0ee4be
Copying config sha256:d9e03d395ae0377c6d8d2cc8a279c365b33773d26603d8b328b2aed85759b4fc
Writing manifest to image destination
Storing signatures


Removed login credentials for all registries
[INFO] Completes setting << container_uri >> in solution_metadata.yaml: 
042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cis-dev/cis-workspace-user/customer-index/train


----

#### **STEP-5**. Training 에 사용될 User Parameters 설정

> solution_metadata.yaml에 사용자 파라미터 및 artifacts 저장 경로를 넣어줍니다. 

In [10]:
## user parameter 입력
registerer.set_candidate_parameters() # todo: selected params?
# artifact 저장 경로 지정
# artifacts_s3_url : full s3 url
registerer.set_airflow_artifacts_uri(
    artifacts_s3_url= f"s3://{S3_BUCKET}/{TRAIN_ARTIFACTS_PREFIX}"
)

{'asset_source': [{'train_pipeline': [{'source': {'branch': 'tabular_dev_solution', 'code': 'http://mod.lge.com/hub/smartdata/ml-framework/alov2-module/input.git', 'requirements': ['pandas==1.5.3']}, 'step': 'input'}, {'source': {'branch': 'prep_dev_solution', 'code': 'http://mod.lge.com/hub/dxadvtech/assets/preps.git', 'requirements': ['requirements.txt']}, 'step': 'preprocess'}, {'source': {'branch': 'tcr_dev_solution', 'code': 'http://mod.lge.com/hub/dxadvtech/assets/tcr.git', 'requirements': ['requirements.txt', 'numpy==1.25.2 --force-reinstall']}, 'step': 'train'}, {'source': {'branch': 'output_dev', 'code': 'http://mod.lge.com/hub/dxadvtech/assets/output.git', 'requirements': ['requirements.txt']}, 'step': 'output'}]}, {'inference_pipeline': [{'source': {'branch': 'tabular_dev_solution', 'code': 'http://mod.lge.com/hub/smartdata/ml-framework/alov2-module/input.git', 'requirements': ['pandas==1.5.3']}, 'step': 'input'}, {'source': {'branch': 'prep_dev_solution', 'code': 'local', '

----

#### **STEP-6**. <del>Training 에 사용될 Cloud resource 선택</del> use clm requests

> solution_metadata.yaml 구성을 위해 그대로 실행 

In [11]:
#----------------------------------------#
#  자원, schedule 은 clm 으로 interface    #
#----------------------------------------#
train_resource = 'standard'
registerer.set_resource(train_resource)


[train] Success updating << resource >> in the solution_metadata.yaml


----

#### **STEP-7**. Inference 용 Sample Data 등록

In [12]:
###################
pipeline = "inference"
###################
# inference pipeline을 solution_metadata.yaml에 추가하고, 현재 registerer의 pipeline type을 inference로 변경 
registerer.append_pipeline(pipeline)


<< solution_metadata.yaml >> updated. - appended pipeline: inference


In [13]:
INFER_DATA_PREFIX = f'data/{SOLUTION_NAME}/inference_data/' # must starts with : 'data/ ... '  
INFER_ARTIFACTS_PREFIX = f'data/{SOLUTION_NAME}/inference_artifacts/' # must starts with : 'data/ ... '

In [14]:
assert INFER_DATA_PREFIX.startswith('data'), ValueError("prefix needs to be start with data/")
assert INFER_ARTIFACTS_PREFIX.startswith('data'), ValueError("prefix needs to be start with data/")
# data prefix 등록
registerer.set_s3_directly(bucket_name=S3_BUCKET,
                           prefix=INFER_DATA_PREFIX)

# s3 데이터 업로드
# 이전에 이미 해당 s3 경로에 존재하던 데이터는 지워집니다.
registerer.s3_upload_data()

# inference results / artifacts prefix 등록
registerer.set_s3_directly(bucket_name=S3_BUCKET,
                           prefix=INFER_ARTIFACTS_PREFIX)
# s3 데이터 업로드
# 이전에 이미 해당 s3 경로에 존재하던 데이터는 지워집니다.
registerer.s3_upload_artifacts()


# model 저장, model.tar.gz 는
# ai conductor 는 inference pipeline 일 때 upload model 의 경로는 train s3 를 참조하기 때문에
# file hierarchy convention 차원에서 s3 prefix 를 TRAIN 으로 바꿈
# train artifact 저장과 다른 이유는 train artifact는 이미 모델을 포함 하고 있음.
registerer.set_s3_directly(bucket_name=S3_BUCKET,
                            prefix=TRAIN_ARTIFACTS_PREFIX)
registerer.s3_upload_model_for_inference() # 대신 pipeline (self.pipeline 은 inference 에 머물러야 함)


[INFO] AWS S3 access check: OK

[INFO] Start uploading << data >> into S3 from local folder:
 /home/jovyan/airflow_test/alo2.2_tcr/input/inference/

[INFO] Deleted pre-existing S3 object: data/customer-index/inference_data/

[INFO] Deleted pre-existing S3 object: data/customer-index/inference_data/inference_data/test/iris.csv

Success uploading into S3: 
s3-an2-cis-dev-data/data/customer-index/inference_data/inference_data/test/iris.csv

Success updating solution_metadata.yaml - << dataset_uri >> info. / pipeline: inference

[INFO] AWS S3 access check: OK

[INFO] Start uploading << inference artifacts >> into S3 from local folder:
 /home/jovyan/airflow_test/alo2.2_tcr/.TEMP_ARTIFACTS_PATH/

[INFO] Deleted pre-existing S3 object: data/customer-index/inference_artifacts/

[INFO] Deleted pre-existing S3 object: data/customer-index/inference_artifacts/inference_artifacts.tar.gz

Success uploading into S3: 
s3-an2-cis-dev-data/data/customer-index/inference_artifacts/inference_artifacts.tar

----

#### **STEP-8**. Inference 용 Docker Container 제작

> DOCKERFILE을 셋팅합니다. \
> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [15]:
# DOCKERFILE setting
registerer.set_docker_contatiner()

Success DOCKERFILE setting. 
 - pipeline: inference


In [16]:
## ECR 등록
## buildah login 실행 (docker in docker)

INFER_ECR_URI = os.path.join(ECR_BASE_URL,SOLUTION_NAME,'inference')

tags = [
    "Key=Company,Value=LGE",
    "Key=Owner,Value=IC360",
    "Key=HQ,Value=CDO",
    "Key=Division,Value=CDO",
    "Key=Infra Region,Value=KIC",
    "Key=Service Mode,Value=DE",
    "Key=Cost Type,Value=COMPUTING",
    "Key=Project,Value=CIS",
    "Key=Sub Project,Value=CISM",
    "Key=System,Value=AIDX"
]
registerer.set_ecr_directly(
    ecr_uri=INFER_ECR_URI,
    docker=False,
    tags=tags
)

p1:  ['aws', 'ecr', 'get-login-password', '--region', 'ap-northeast-2']
[INFO] target AWS ECR url: 
042969618971.dkr.ecr.ap-northeast-2.amazonaws.com
p2:  ['sudo', 'buildah', 'login', '--username', 'AWS', '--password-stdin', '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com']
[INFO] AWS ECR | docker login result: 
 Login Succeeded!

[INFO] Target AWS ECR repository: 
ecr-repo-an2-cis-dev/cis-workspace-user/customer-index/inference
Failed to AWS ECR create-repository:
 + Command '['aws', 'ecr', 'create-repository', '--region', 'ap-northeast-2', '--repository-name', 'ecr-repo-an2-cis-dev/cis-workspace-user/customer-index/inference', '--image-scanning-configuration', 'scanOnPush=true', '--tags', 'Key=Company,Value=LGE', 'Key=Owner,Value=IC360', 'Key=HQ,Value=CDO', 'Key=Division,Value=CDO', 'Key=Infra Region,Value=KIC', 'Key=Service Mode,Value=DE', 'Key=Cost Type,Value=COMPUTING', 'Key=Project,Value=CIS', 'Key=Sub Project,Value=CISM', 'Key=System,Value=AIDX']' returned non-zero exit statu

> Build docker \
> AWS ECR에 docker 이미지를 push 합니다. \
> solution_metadata.yaml에 container uri를 넣어줍니다.

In [17]:
# docker build 
registerer.build_docker()
registerer.docker_push()
registerer.set_container_uri() 

STEP 1/17: FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye
STEP 2/17: RUN apt-get update
Get:1 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:2 http://deb.debian.org/debian-security bullseye-security InRelease [48.4 kB]
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Get:4 http://deb.debian.org/debian bullseye/main amd64 Packages [8068 kB]
Get:5 http://deb.debian.org/debian-security bullseye-security/main amd64 Packages [269 kB]
Get:6 http://deb.debian.org/debian bullseye-updates/main amd64 Packages [18.8 kB]
Fetched 8564 kB in 1s (7996 kB/s)
Reading package lists...
STEP 3/17: RUN apt-get install -y apt-utils
Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  apt-utils
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 439 kB of archives.
After this operation, 1192 kB of additional disk space will be used.
Get:1 http://deb.debian.org

debconf: delaying package configuration, since apt-utils is not installed


Reading package lists...
Building dependency tree...
Reading state information...
ca-certificates is already the newest version (20210119).
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu bzip2 containerd cpp
  cpp-10 dmsetup dpkg-dev g++ g++-10 gcc-10 git-man iptables libasan6
  libatomic1 libbinutils libbrotli1 libc-dev-bin libc6-dev libcc1-0
  libcrypt-dev libctf-nobfd0 libctf0 libcurl3-gnutls libdevmapper1.02.1
  libdpkg-perl liberror-perl libgcc-10-dev libgdbm-compat4 libgomp1 libip4tc2
  libip6tc2 libisl23 libitm1 libjq1 libldap-2.4-2 liblsan0 libmnl0 libmpc3
  libmpfr6 libncurses6 libnetfilter-conntrack3 libnfnetlink0 libnftnl11
  libnghttp2-14 libnsl-dev libonig5 libperl5.32 libprocps8 libpsl5
  libquadmath0 librtmp1 libsasl2-2 libsasl2-modules-db libssh2-1
  libstdc++-10-dev libtirpc-dev libtsan0 libubsan1 libxtables12 linux-libc-dev
  make patch perl perl-modules-5.32 runc tini xz-utils
Suggested packages:
  binutils-d

debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.32.1 /usr/local/share/perl/5.32.1 /usr/lib/x86_64-linux-gnu/perl5/5.32 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl-base /usr/lib/x86_64-linux-gnu/perl/5.32 /usr/share/perl/5.32 /usr/local/lib/site_perl) at /usr/share/perl5/Debconf/FrontEnd/Readline.pm line 7, <> line 76.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 141 MB in 1s (184 MB/s)
Selecting previously unselected package perl-modules-5.32.
(Reading database ... 7121 files and directories currently installed.)
Preparing to unpack .../00-perl-modules-5.32_5.32.1-4+deb11u3_all.deb ...
Unpacking perl-modules-5.32 (5.32.1-4+deb11u3) ...
Selecting previously unselected package libgdbm-compat4:amd64.
Preparing to unpack .../01-libgdbm-compat4_1.19-2_amd64.deb ...
Unpacking libgdbm-compat4:amd64 (1.19-2) ...
Selecting previously unselected package libperl5.32:amd64.
Preparing to unpack .../02-libperl5.32_5.32.1-4+deb11u3_amd64.deb ...
Unpacking libperl5.32:amd64 (5.32.1-4+deb11u3) ...
Selecting previously unselected package perl.
Preparing to unpack .../03-perl_5.32.1-4+deb11u3_amd64.deb ...
Unpacking perl (5.32.1-4+deb11u3) ...
Selecting previously unselected package dmsetup.
Preparing to unpack .../04-dmsetup_2%3a1.02.175-2.1_amd64.deb ...
Unpacking dmsetup (2:1.02.175-2.1) ...
Selecting previously unselected package libdevmapper1.02.1:a

...

Getting image source signatures
Copying blob sha256:0baf2321956a506afcddaafe217bc852e4c56a9640530b1b2f98b3378d4b6173
Copying blob sha256:e00d2ab597e7579cfd8251f047918d0bbfbd07230dad84a2f2a863fa0b5de84d
Copying blob sha256:2c73ee9daea8b487fd2035729eb8dc53c69d46dcab0b0a45114fe83a788a7faf
Copying blob sha256:b8ef0148a6d0f9aefbe116049b8a263aca65af0fa7c52072da94ef4434ca6263
Copying blob sha256:f9c504e80eedd8d7bd1fb3cfc88fcbc51b0f154844cae65d26939511f4a60d51
Copying blob sha256:1163ceb973e52a01a480de248c59b1eb9ac9ade44430d938393d141b74ea47ff
Copying config sha256:5b6c0ca1084e683849eec3aa3ced06ca588da8f76dffbcfdc2ac6265b824dfe9
Writing manifest to image destination
Storing signatures


--> 5b6c0ca1084
Successfully tagged 042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cis-dev/cis-workspace-user/customer-index/inference:latest
5b6c0ca1084e683849eec3aa3ced06ca588da8f76dffbcfdc2ac6265b824dfe9


Getting image source signatures
Copying blob sha256:2c73ee9daea8b487fd2035729eb8dc53c69d46dcab0b0a45114fe83a788a7faf
Copying blob sha256:b8ef0148a6d0f9aefbe116049b8a263aca65af0fa7c52072da94ef4434ca6263
Copying blob sha256:0baf2321956a506afcddaafe217bc852e4c56a9640530b1b2f98b3378d4b6173
Copying blob sha256:e00d2ab597e7579cfd8251f047918d0bbfbd07230dad84a2f2a863fa0b5de84d
Copying blob sha256:f9c504e80eedd8d7bd1fb3cfc88fcbc51b0f154844cae65d26939511f4a60d51
Copying blob sha256:1163ceb973e52a01a480de248c59b1eb9ac9ade44430d938393d141b74ea47ff
Copying config sha256:5b6c0ca1084e683849eec3aa3ced06ca588da8f76dffbcfdc2ac6265b824dfe9


Removed login credentials for all registries
[INFO] Completes setting << container_uri >> in solution_metadata.yaml: 
042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cis-dev/cis-workspace-user/customer-index/inference


Writing manifest to image destination
Storing signatures


----

#### **STEP-9**. Inference 용 User Parameters 제작

> solution_metadata.yaml에 사용자 파라미터, artifacts 경로 등을 넣어줍니다. \
> solution_metadata.yaml에 model 경로를 넣어줍니다. <u>(추론 시에만 필요합니다.)</u>

In [18]:
# 아래는 configuration 생성을 위해 생성 (solution_metadata.yaml )
registerer.set_candidate_parameters()

registerer.set_airflow_artifacts_uri(
    artifacts_s3_url= f"s3://{S3_BUCKET}/{INFER_ARTIFACTS_PREFIX}"
)

registerer.set_airflow_model_url(
    train_artifact_s3_url=f"s3://{S3_BUCKET}/{TRAIN_ARTIFACTS_PREFIX}"
) # 참고: model은 train artifacts 경로에 존재하기로 약속..

{'asset_source': [{'train_pipeline': [{'source': {'branch': 'tabular_dev_solution', 'code': 'http://mod.lge.com/hub/smartdata/ml-framework/alov2-module/input.git', 'requirements': ['pandas==1.5.3']}, 'step': 'input'}, {'source': {'branch': 'prep_dev_solution', 'code': 'http://mod.lge.com/hub/dxadvtech/assets/preps.git', 'requirements': ['requirements.txt']}, 'step': 'preprocess'}, {'source': {'branch': 'tcr_dev_solution', 'code': 'http://mod.lge.com/hub/dxadvtech/assets/tcr.git', 'requirements': ['requirements.txt', 'numpy==1.25.2 --force-reinstall']}, 'step': 'train'}, {'source': {'branch': 'output_dev', 'code': 'http://mod.lge.com/hub/dxadvtech/assets/output.git', 'requirements': ['requirements.txt']}, 'step': 'output'}]}, {'inference_pipeline': [{'source': {'branch': 'tabular_dev_solution', 'code': 'http://mod.lge.com/hub/smartdata/ml-framework/alov2-module/input.git', 'requirements': ['pandas==1.5.3']}, 'step': 'input'}, {'source': {'branch': 'prep_dev_solution', 'code': 'local', '

----

#### **STEP-10**. <del>Inference 용 Resource 선택</del>, use clm requests

> solution_metadata.yaml 구성을 위해 그대로 실행 

In [19]:
#----------------------------------------#
#  자원, schedule 은 clm 으로 interface    #
#----------------------------------------#
inference_resource = 'standard'
registerer.set_resource(inference_resource)


[inference] Success updating << resource >> in the solution_metadata.yaml


----

#### **STEP-11**. <del>Description 추가</del> Just for solution_metadata.yaml

> solution_metadata.yaml에 description을 추가합니다.

In [20]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
USER_DESCRITPION ={
    'title': "EdgeConductor UI Solution Title", 
        
    'overview': "AI Advisor Test",

    'input_data': "Test input data",
    
    'output_data': "Test output data",

    'user_parameters': "Test params",
    
    'algorithm': "Tabular Classification"
}
#----------------------------------------#

registerer.set_description(USER_DESCRITPION)


<< solution_metadata.yaml >> updated. 
- solution metadata description:
 {'icon': 's3://s3-an2-cis-dev-data/icons/customer-index/icon.svg', 'title': 'EdgeConductor UI Solution Title', 'overview': 'AI Advisor Test', 'input_data': 's3-an2-cis-dev-dataTest input data', 'output_data': 's3-an2-cis-dev-dataTest input data', 'user_parameters': 'Test params', 'algorithm': 'Tabular Classification'}


----

#### **STEP-12**. Edge 관련 YAML key-value 추가, Just for solution_metadata.yaml

In [21]:
# release-2.1.3에서는 wrangler 미지원 
# wrangler 정보 등록 
registerer.set_wrangler()

In [24]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
# EdgeConductor관련 정보 등록
edgeconductor_interface = {
            # 모델 재학습 시 re-labeling이 가능한 컨텐츠 인지 
            'support_labeling': True,
            # EdgeConductor UI에서 보여질 추론 결과 데이터 타입 
            'inference_result_datatype': 'table', # 'image' / 'table'
            # 모델 재학습 시 입력으로 들어가는 데이터 타입 
            'train_datatype': 'table' # 'image' / 'table'
        }

#----------------------------------------#
registerer.set_edge(edgeconductor_interface)

----